In [75]:
# Installing the mysql-connector-python library to establish conneciton to MySQL server.
!pip install mysql-connector-python

In [76]:
import mysql.connector
from mysql.connector import errorcode
from time import time
import csv
import gc

In [77]:
class app_data:
    def __init__(self, name, roll):
        self.name = name
        self.roll = roll

In [78]:
# This funciton was written to extract a report tuple for each query been executed, a .csv file will be created
# And report parameters such as, execution time, throughputs will be written in there.
def write_report(active_db, report_tuple_list):
    with open('./Rep_'+ active_db +'.csv', 'w', newline='') as csvfile:
        reportwriter = csv.writer(csvfile, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        reportwriter.writerow(['Start_s', 'End_s', 'Execution_s', 'Throughput', 'Query'])
        
        for index in range(len(report_tuple_list)):
            report_tuple = report_tuple_list[index];
            reportwriter.writerow([report_tuple['start_time_s'], report_tuple['end_time_s'], report_tuple['execution_time_s'], report_tuple['throughput'], report_tuple['query_string']])

In [79]:
def query_measure(cnx, query_string):
    
    # A datatype of dictionary is created to keep track of the status of each query been executed
    report_tuple = {
        'query_string': query_string,
        'start_time_s': '',
        'end_time_s': '',
        'execution_time_s': '',
        'throughput': ''
    }
    
    # if the connection established
    if cnx and cnx.is_connected():
        with cnx.cursor(buffered=True) as cursor:
            print(report_tuple['query_string'])
            
            # del report_tuple['query_result']
            gc.collect()

            # Measure the time before query execution 
            report_tuple['start_time_s'] = time();
            report_tuple['query_result'] = cursor.execute(report_tuple['query_string'])
            
            # Measure the time after successfully query execution 
            report_tuple['end_time_s'] = time();
            
            # Calculates the execution time and throughput
            report_tuple['execution_time_s'] = report_tuple['end_time_s'] - report_tuple['start_time_s']
            report_tuple['throughput'] = 1 / (report_tuple['end_time_s'] - report_tuple['start_time_s'])
            
            return report_tuple;
            
            # rows = cursor.fetchall()
            # for rows in rows:
                # print(rows)
        cnx.close()
    else:
        print("Could not connect")
    

In [80]:
db_dict = {
    '3k': 'cs505_prj_tpch_3k',
    '12k': 'cs505_prj_tpch_12k',
    '60k': 'cs505_prj_tpch_60k',
    '120k': 'cs505_prj_tpch_120k'
}

for key, value in db_dict.items():
    active_db = value
    # print("Current Database: " + str(value))
    # key_value = "(key, value) = ({key}, {value})".format(key = key, value = value)
    # print(key_value + str("\n"))

    config = {
      'user': 'root',
      'password': '********',
      'host': '127.0.0.1',
      'database': active_db,
      'raise_on_warnings': True
    }
    report_tuple_list = [];
    
    for x in range(1, 23):
        current_query = "SELECT * FROM {db_name}.q{query_id}".format(db_name = active_db, query_id = x)
        
        do_proceed = "yes"
        # if(5 < x < 10 ):
            # do_proceed = "yes"
        
        if(do_proceed == "yes"):
            try:
                cnx = mysql.connector.connect(**config)
                report_tuple = query_measure(cnx, current_query)
                
                print(report_tuple['execution_time_s'])
                print(report_tuple['start_time_s'])
                print(report_tuple['end_time_s'])

                print_str = "(ExecutionTime = EndTime - StartTime) = ({ExecutionTime} = {EndTime} - {StartTime})".format(ExecutionTime = report_tuple['execution_time_s'], EndTime = report_tuple['end_time_s'], StartTime = report_tuple['start_time_s']);

                report_tuple_list.append(report_tuple)


                print(print_str)

            except mysql.connector.Error as err:
              if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
                print("Something is wrong with your user name or password")
              elif err.errno == errorcode.ER_BAD_DB_ERROR:
                print("Database does not exist")
              else:
                print("Encounter Error: " + str(err))
                
    write_report(active_db, report_tuple_list)    
    

SELECT * FROM cs505_prj_tpch_120k.q1
0.3515164852142334
1713409321.9327674
1713409322.2842839
(ExecutionTime = EndTime - StartTime) = (0.3515164852142334 = 1713409322.2842839 - 1713409321.9327674)
SELECT * FROM cs505_prj_tpch_120k.q2
126.6253809928894
1713409322.5392294
1713409449.1646104
(ExecutionTime = EndTime - StartTime) = (126.6253809928894 = 1713409449.1646104 - 1713409322.5392294)
SELECT * FROM cs505_prj_tpch_120k.q3
1.7814183235168457
1713409449.406028
1713409451.1874464
(ExecutionTime = EndTime - StartTime) = (1.7814183235168457 = 1713409451.1874464 - 1713409449.406028)
SELECT * FROM cs505_prj_tpch_120k.q4
0.23008513450622559
1713409451.4547217
1713409451.6848068
(ExecutionTime = EndTime - StartTime) = (0.23008513450622559 = 1713409451.6848068 - 1713409451.4547217)
SELECT * FROM cs505_prj_tpch_120k.q5
3.7121217250823975
1713409451.921588
1713409455.6337097
(ExecutionTime = EndTime - StartTime) = (3.7121217250823975 = 1713409455.6337097 - 1713409451.921588)
SELECT * FROM cs505